<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send metrics analysis by email with chat plugin CTA

**Tags:** #linkedin #tool #posts #engagement #metrics #analytics #automation #email #naas #notification #chat #plugin

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Last update:** 2023-05-29 (Created: 2022-05-11)

**Description:** This notebook automates the process of sending engagement metrics on LinkedIn via email and the generation of a Naas Chat Plugin.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
import os
import json
import random
import time
import pandas as pd
from datetime import datetime, date
from dateutil.parser import parse
import naas 
import naas_drivers
from naas_drivers import emailbuilder, linkedin
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.graph_objects as go
try:
    import tiktoken
except:
    !pip install tiktoken --user
    import tiktoken

### Setup variables
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# Define scenario 
is_demo = True # Default to True

# For LinkedIn 
limit = 2 #number of post retrieved
refresh_interval = 2 #time in minutes between two LinkedIn queries
LI_AT = naas.secret.get("LINKEDIN_LI_AT")  # EXAMPLE : "AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID")   # EXAMPLE : "ajax:8379907400220387585"
PROFILE_URL = naas.secret.get("LINKEDIN_PROFILE_URL")  # EXAMPLE "https://www.linkedin.com/in/myprofile/"

# For Emails
EMAIL_TO = "jeremy@naas.ai"  # you will receive weekly summary at this email
EMAIL_FROM = None  # summary will have this email as sender. Only available for your naas email, otherwise you will receive this email from notification@naas.ai
EMAIL_SUBJECT = (f"📲 Content Engine - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
DATE_FORMAT = "%Y-%m-%d"

# For AI Chat Plugin
plugin_name = "📲 Content Assistant"
plugin_model = "gpt-3.5-turbo-16k"
plugin_temperature = 0
plugin_max_tokens = 8192
system_prompt_max_tokens = 2084

# For Asset Generation
output_dir = "/home/ftp/__abi__/outputs/by_modules/content_analytics"
csv_file_name = "content_analytics_data.csv"
image_file_name = "content_analytics_views.png"
plugin_file_name = "content_analytics_plugin.json"

## Model

### Setup directories

In [ ]:
# Check if directory exists and create it if not
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# Generate outputs files path
csv_file_path = os.path.join(output_dir, csv_file_name)
image_file_path = os.path.join(output_dir, image_file_name)
plugin_file_path = os.path.join(output_dir, plugin_file_name)
print('📂 CSV file path:', csv_file_path)
print('📂 Image file path:', image_file_path)
print('📂 Plugin file path:', plugin_file_path)

### Get content data

In [ ]:
def get_last_posts(
    li_at,
    JSESSIONID,
    linkedin_url,
    limit,
    file_path,
    refresh_interval,
    demo_mode=is_demo  
):
    # Init
    df = pd.DataFrame()
    update_data = True
    
    # Check if demo_mode is True
    if demo_mode:
        file_path = "/home/ftp/__abi__/inputs/demo_data/demo_data-linkedin_post.csv"
        if os.path.exists(file_path):
            # Read file
            df = pd.read_csv(file_path)
            df.to_csv(csv_file_path, index=False)
            return df
    
    # Check if output already exists
    if os.path.exists(csv_file_path):
        # Read file
        df = pd.read_csv(csv_file_path)
        
        # Assess if LinkedIn API can be invoked based on the last call.
        # To emulate human interaction, we must avoid making excessive calls to the LinkedIn API. Overdoing this could result in being banned.
        if len(df) > 0:
            if "DATE_EXTRACT" in df.columns:
                # Manage calls to API
                last_update_date = df.loc[0, "DATE_EXTRACT"]
                time_last_update = datetime.now() - datetime.strptime(last_update_date, "%Y-%m-%d %H:%M:%S")
                minute_last_update = time_last_update.total_seconds() / 60
                if minute_last_update < refresh_interval:
                    update_data = False
                    print(f"ℹ️ Nothing to update as the last retrieve was {int(minute_last_update)} minutes ago.")
                
    if update_data:
        # Get last posts
        df = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(linkedin_url, limit=limit)
        # Save last posts in CSV
        df.to_csv(csv_file_path, index=False)
        print("💾 Dataframe successfully saved:", csv_file_path)
    return df

# Call the function and display results
df_posts = get_last_posts(LI_AT, JSESSIONID, PROFILE_URL, limit, csv_file_path, refresh_interval)
print("✅ Row fetched:", len(df_posts))
df_posts

### Create asset from data

In [ ]:
# Share output with naas
data_link = naas.asset.add(csv_file_path)  # Assuming you have the relevant naas asset module

### Prepare data for analysis

In [ ]:
def get_posts(file_path):
    try:
        df = pd.read_csv(csv_file_path)
        # Convert 'PUBLISHED_DATE' to datetime format if it's not already
        if df['PUBLISHED_DATE'].dtype == 'object':
            df['PUBLISHED_DATE'] = pd.to_datetime(df['PUBLISHED_DATE'])
        # Sort by 'PUBLISHED_DATE' and get the last 10 rows
        df = df.sort_values('PUBLISHED_DATE').tail(10)
    except FileNotFoundError as e:
        # Empty dataframe returned
        return pd.DataFrame()
    return df
df = get_posts(csv_file_path)
df

In [ ]:
df['DATE'] = df['PUBLISHED_DATE'].dt.strftime('%Y-%m-%d')
df['TIME'] = df['PUBLISHED_DATE'].dt.strftime('%H:%M:%S')
df

### Create chart

In [ ]:
def create_vertical_barchart(df,
                               label="DATE",
                               value="VIEWS",
                               value_d="VIEWS"):
    # Init
    fig = go.Figure()
    
    # Return empty fig if dataframe is empty
    if len(df) == 0:
        return fig
    
    # Create fig
    fig.add_trace(
        go.Bar(
            y=df[value],
            x=df[label],
            text=df[value_d],
            textposition="outside",
            marker=dict(color="#0f93d2"),
            orientation="v"
        )
    ) 
    
    fig.update_traces(showlegend=False)
    
    # Plotly: Create title
    total_value = "{:,.0f}".format(df[value].sum()).replace(",", " ")
    title = f"<b><span style='font-size: 20px;'>Past {limit} Content Views</span></b><br><span style='font-size: 18px;'>Total KPI: {total_value}</span>"
    fig.update_layout(
        title=title,
        #title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        #margin_r=10,
        #margin_l=10,
    )
    
    fig.update_xaxes(showticklabels=True)
    
    return fig

create_vertical_barchart(df)

### Create asset from chart

In [ ]:
# Create the chart
chart = create_vertical_barchart(df)

# Save as PNG
pio.write_image(chart, image_file_path)

#graph_url = naas.asset.add("chart.html", {"inline": True})
graph_image = naas.asset.add(image_file_path)

### Set email parameters

In [ ]:
today = datetime.now().strftime(DATE_FORMAT)
today

In [ ]:
likes = df.LIKES.sum()
views = df.VIEWS.sum()
shares = df.SHARES.sum()
comments = df.COMMENTS.sum()

# preview of what will be send by email:
print("This week's cumulative statistics:")
print(
    "\n\t👀 Impressions\t",
    views,
    "\n\t👍 Likes\t",
    likes,
    "\n\t💬 Comments\t",
    comments,
    "\n\t⏩ Shares\t",
    shares,
)

### Create Naas Chat plugin

In [ ]:
system_prompt = f"""Act as a Content Agent who has access to valuable data and insights about the content strategy. 
Your role is to manage and optimize the content, ensuring it reaches the target audience effectively. 
You will need to analyze user behavior and preferences to understand their needs and interests. 
Utilize the data from various sources to personalize the content strategy and track its performance. 
Your ultimate goal is to create engaging and relevant content that resonates with the audience, contributing to the overall success of the content strategy. 
The first message should be about presenting yourself with a maximum of 5 bullet points and displaying the current content analytics data to be displayed as an image inside the markdown of the chat: {graph_image}. 
Then, wait for the first answer from the user, and then start with the first high-level analysis. 
Here is the data from the content analytics that you should focus on: {df}

"""
system_prompt

### Check token count 

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

system_prompt_tokens = num_tokens_from_string(system_prompt, "cl100k_base")
if system_prompt_tokens > system_prompt_max_tokens:
    print("⚠️ Be carefull, your system prompt looks too big. Tokens:", system_prompt_tokens)
else:
    print("✅ System prompt tokens count OK:", system_prompt_tokens)

### Generate plugin

In [ ]:
# Create json
plugin = {
    "name": plugin_name,
    "model": plugin_model,
    "temperature": plugin_temperature,
    "max_tokens": plugin_max_tokens,
    "prompt": system_prompt,
}

# Save dict to JSON file
with open(plugin_file_path, "w") as f:
    json.dump(plugin, f)
print("💾 Plugin successfully saved:")

plugin = naas.asset.add(plugin_file_path, params={"inline": True})

### Generate email content

In [ ]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f}"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [ ]:
def email_brief(
    today,
    views,
    likes,
    comments,

):
    content = {
        'title': ("📲 Content Engine - Email Update"),
        'heading': (f"Date:{today}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your content engine email as of {today}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(graph_image),
        "text_1": emailbuilder.text(
            f"On the past {limit} pieces of content, your reach a total of:"
        ),
        "list_1": emailbuilder.list(
            [
                f"Views : <b>{format_number(views)}</b>",
                f"Likes : <b>{format_number(likes)}</b>",
                f"Comments : <b>{format_number(comments)}</b>",
            ]
        ),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin}"),
            text="Start Chatting With Agent",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    today,
    views,
    likes,
    comments,
)

## Output

### Send email notification

In [ ]:
# sends the email
naas.notification.send(
    email_to=EMAIL_TO, subject=EMAIL_SUBJECT, html=email_content, email_from=EMAIL_FROM
)